### Notebook to load and analyze Federal Court cases

Sean Rehaag

License: Creative Commons Attribution-NonCommercial 4.0 International [(CC BY-NC 4.0)](https://creativecommons.org/licenses/by-nc/4.0/). NOTE: Users must also comply with upstream [licensing](https://www.fct-cf.gc.ca/en/pages/important-notices) for the FC data source, as well as requests on source urls not to allow indexing of the documents by search engines to protect privacy. As a result, users must not make the data available in formats or locations that can be indexed by search engines.

Dataset & Code to be cited as: 

    Sean Rehaag, "Federal Court Bulk Decisions Dataset" (2023), online: Refugee Law Laboratory <https://refugeelab.ca/bulk-data/fc/>.

Notes:

(1) Data Source: [Federal Court](https://www.fct-cf.gc.ca). 

(2) Unofficial Data: The data are unofficial reproductions of materials on the Federal Court website. Links to official versions are included in the dataset.

(3) Non-Affiliation / Endorsement: The data has been collected and reproduced without any affiliation or endorsement from the Federal Court.

(4) Non-Commerical Use: As indicated in the license, data may be used for non-commercial use (with attribution) only. For commercial use, see the Federal Court of Appeal website's [Terms of Use](https://www.fct-cf.gc.ca/en/pages/important-notices).

(5) Accuracy: Data was collected and processed programmatically for the purposes of academic research. While we make best efforts to ensure accuracy, data gathering of this kind inevitably involves errors. As such the data should be viewed as preliminary information aimed to prompt further research and discussion, rather than as definitive information. 

(6) Limitation: Only includes cases with neutral citation, which began to be used in 2001

(7) Delay: Decisions may take many months to be translated (sometimes over a year). As a result, in the most recent years, decisions may only be available in one language.

### Requirements:

    pip install pandas

### If using parquet

    pip install pyarrow

### if loading remotely (other than via Hugging Face)
    
    pip install requests

### If loading remotely via Hugging Face

    pip install datasets
    

(Written on Python 3.9.12)

### Load Data

Four Options: Local & Remote

In [5]:
# OPTION 1: Load Hugging Face dataset

from datasets import load_dataset
import pandas as pd

dataset = load_dataset("refugee-law-lab/canadian-legal-data", split="train", data_dir="FC")

# convert to dataframe
df = pd.DataFrame(dataset)

df


Found cached dataset parquet (C:/Users/SR/.cache/huggingface/datasets/refugee-law-lab___parquet/refugee-law-lab--canadian-legal-data-b213d05c6af1cba8/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


In [ ]:
# OPTION 2: Load parquet data remotely from Huggingface without cloning repo
import pandas as pd
import requests
from io import BytesIO

url = 'https://huggingface.co/datasets/refugee-law-lab/canadian-legal-data/resolve/main/FC/train.parquet'

# load data
results = requests.get(url)

# convert to dataframe
df = pd.read_parquet(BytesIO(results.content))

df

# (if code fails, add engine='pyarrow' to read_parquet() function)

In [ ]:
# OPTION 3: Load data remotely from GitHub without cloning repo
# Note: load time varies depending on internet connection (approx 1.5 GB of data for all years/languages)
# This is the slowest loading option.

import pandas as pd
import json
import requests

# Set variables
start_year = 2001  # First year of data sought (1877 +)
end_year = 2023  # Last year of data sought (2023 -)
languages_sought = ['en', 'fr']  # languages in list e.g. ['en', 'fr'] or ['en']


base_ulr = 'https://raw.githubusercontent.com/Refugee-Law-Lab/fc_bulk_data/master/DATA/YEARLY/'

# load data
results = []
for year in range(start_year, end_year+1):
    for language in languages_sought:
        url = base_ulr + f'{year}_{language}.json'
        results.extend(requests.get(url).json())

# convert to dataframe
df = pd.DataFrame(results)

df

In [ ]:
# OPTION 4: Load data locally via cloned repo

# First, clone git repo
# Then run this code to load data

import pandas as pd
import json
import pathlib

# Set variables
start_year = 2001  # First year of data sought (1877 +)
end_year = 2023  # Last year of data sought (2023 -)
languages_sought = ['en', 'fr']  # languages in list e.g. ['en', 'fr'] or ['en']

# Set path to data
data_path = pathlib.Path('DATA/YEARLY/')

# load data
results = []
for year in range(start_year, end_year+1):
    for language in languages_sought:
        with open(data_path / f'{year}_{language}.json') as f:
            results.extend(json.load(f))

# convert to dataframe
df = pd.DataFrame(results)

df


### Analyze Data

In [6]:
# View dataframe
df.head()

,citation,citation2,dataset,year,name,language,document_date,source_url,scraped_timestamp,unofficial_text,other
0,2001 FCT 1,,FC,2001,Adecon Ship Management Inc. v. Cuba,en,2001-02-01,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-18,Adecon Ship Management Inc. v. Cuba\nCourt (s)...,
1,2001 FCT 10,,FC,2001,Islam v. Canada (Minister of Citizenship and I...,en,2001-02-02,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-18,Islam v. Canada (Minister of Citizenship and I...,
2,2001 FCT 100,,FC,2001,Duterville v. Canada,en,2001-02-20,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-18,Duterville v. Canada\nCourt (s) Database\nFede...,
3,2001 FCT 1000,,FC,2001,LS Entertainment Group Inc. v. KALOS VISION LT...,en,2001-09-07,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-18,LS Entertainment Group Inc. v. KALOS VISION LT...,
4,2001 FCT 1001,,FC,2001,Ay v. Canada (Minister of Citizenship and Immi...,en,2001-09-07,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2022-08-18,Ay v. Canada (Minister of Citizenship and Immi...,


In [7]:
df.tail()

,citation,citation2,dataset,year,name,language,document_date,source_url,scraped_timestamp,unofficial_text,other
59901,2023 CF 840,,FC,2023,Vill c. Bell Canada,fr,2023-06-14,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2023-07-03,Vill c. Bell Canada\nBase de données – Cour (s...,
59902,2023 CF 853,,FC,2023,Gosselin c. Canada (Procureur général),fr,2023-06-16,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2023-07-03,Gosselin c. Canada (Procureur général)\nBase d...,
59903,2023 CF 861,,FC,2023,Stoica c. Canada (Sécurité puplique et Protect...,fr,2023-06-20,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2023-07-03,Stoica c. Canada (Sécurité puplique et Protect...,
59904,2023 CF 893,,FC,2023,"Voltage Pictures, LLC c. Salna",fr,2023-06-26,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2023-07-03,"Voltage Pictures, LLC c. Salna\nBase de donnée...",
59905,2023 CF 98,,FC,2023,Boloh 1(A) c. Canada,fr,2023-01-20,https://decisions.fct-cf.gc.ca/fc-cf/decisions...,2023-06-27,Boloh 1(A) c. Canada\nBase de données – Cour (...,


In [8]:
# language counts
df['language'].value_counts()

en    30478
fr    29428
Name: language, dtype: int64

In [9]:
# Yearly counts
year_counts = df.year.value_counts()
years_count = sorted(year_counts.index)
for year_count in years_count:
    print(f'{year_count}: {year_counts[year_count]}')


2001: 2806
2002: 2638
2003: 2961
2004: 3506
2005: 3408
2006: 3069
2007: 2735
2008: 2797
2009: 2499
2010: 2578
2011: 2903
2012: 2748
2013: 2285
2014: 2186
2015: 2610
2016: 2480
2017: 2159
2018: 2262
2019: 2691
2020: 2191
2021: 2711
2022: 2861
2023: 822
